In [47]:
import pandas as pd
import numpy as np
movies = pd.read_csv("//kaggle/input/tmdb-movies-daily-updates/TMDB_all_movies.csv")
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088807 entries, 0 to 1088806
Data columns (total 28 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1088807 non-null  int64  
 1   title                    1088795 non-null  object 
 2   vote_average             1088807 non-null  float64
 3   vote_count               1088807 non-null  float64
 4   status                   1088807 non-null  object 
 5   release_date             976797 non-null   object 
 6   revenue                  1088807 non-null  float64
 7   runtime                  1088807 non-null  float64
 8   budget                   1088807 non-null  float64
 9   imdb_id                  624291 non-null   object 
 10  original_language        1088807 non-null  object 
 11  original_title           1088795 non-null  object 
 12  overview                 918584 non-null   object 
 13  popularity               1088807 non-null 

In [49]:
columns = [
    'id',
    'title',
    'overview',
    'original_language',
    'genres',
    'cast',
    'director',
    'writers',
    'release_date',
    'imdb_rating',
    'poster_path'
]

movies = movies[columns]
movies = movies[movies['original_language']=='en']

In [50]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 540526 entries, 2 to 1088806
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 540526 non-null  int64  
 1   title              540515 non-null  object 
 2   overview           528183 non-null  object 
 3   original_language  540526 non-null  object 
 4   genres             365913 non-null  object 
 5   cast               347354 non-null  object 
 6   director           427799 non-null  object 
 7   writers            239991 non-null  object 
 8   release_date       469923 non-null  object 
 9   imdb_rating        229634 non-null  float64
 10  poster_path        381452 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 49.5+ MB


In [51]:
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies = movies[movies['release_date']<='2024-12-31']

In [52]:
movies.dropna(inplace = True)

In [54]:
movies['tags'] = movies['overview']+movies['genres']+movies['cast']+movies['director']+movies['writers']

In [72]:
train_df = movies[['id','title','tags','imdb_rating','release_date','poster_path']]

In [73]:
train_df.shape

(119695, 6)

In [74]:
train_df['tags'] = train_df['tags'].apply(lambda x: x.lower())

/tmp/ipykernel_148/3851114238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['tags'] = train_df['tags'].apply(lambda x: x.lower())


In [75]:
train_df.head()

,id,title,tags,imdb_rating,release_date,poster_path
2,5,Four Rooms,it's ted the bellhop's first night on the job....,6.7,1995-12-09,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg
3,6,Judgment Night,"four young friends, while taking a shortcut en...",6.6,1993-10-15,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg
6,11,Star Wars,princess leia is captured and held hostage by ...,8.6,1977-05-25,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg
7,12,Finding Nemo,"nemo, an adventurous young clownfish, is unexp...",8.2,2003-05-30,/eHuGQ10FUzK1mdOY69wF5pGgEf5.jpg
8,13,Forrest Gump,a man with a low iq has accomplished great thi...,8.8,1994-06-23,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg


In [76]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english')

In [77]:
vectors = cv.fit_transform(train_df['tags']).toarray()

In [78]:
cv.get_feature_names_out()

array(['000', '10', '100', ..., 'zombie', 'zombies', 'zoë'], dtype=object)

In [79]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  # 6 includes the movie itself
model.fit(vectors)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [82]:
def recommend(movie_title):
    # Get index of the movie
    index = train_df[train_df['title'] == movie_title].index[0]
    
    # Find nearest neighbors
    distances, indices = model.kneighbors(vectors[index:index+1], n_neighbors=6)
    
    # Print top 5 similar (excluding the movie itself)
    print(f"Top 5 movies similar to '{movie_title}':")
    for i in indices[0][1:]:  # skip index[0] (the movie itself)
        print(train_df.iloc[i][['title','release_date']])

In [83]:
recommend("Forrest Gump")

Top 5 movies similar to 'Forrest Gump':
title                          Home
release_date    2019-10-18 00:00:00
Name: 437163, dtype: object
title             The Sarah Vaccine
release_date    2021-05-07 00:00:00
Name: 588638, dtype: object
title            Sins of the Father
release_date    2022-10-22 00:00:00
Name: 630290, dtype: object
title                 It's Not Over
release_date    2022-11-30 00:00:00
Name: 739453, dtype: object
title             No One Would Tell
release_date    2018-09-16 00:00:00
Name: 382178, dtype: object
